Training a Convolutional Neural Network
----------------------------------------------------

In this notebook we train a simple convolutional neural network to classify handwritten digits (**MNIST**) using Tensorflow's low-level API 

#### Import  required libraries

In [1]:
import tensorflow as tf
import numpy as np

#### Build the convNet

CNN architecture: two conv layers, one pool layer and two fully connected layers

In [2]:
# MNIST image
height = 28
width = 28
channels = 1
n_inputs = height * width

# conv layer 1
c1_n_fmaps = 32
c1_kernel_size = 3
c1_strides = 1
c1_padding = 'SAME'

# conv layer 2
c2_n_fmaps = 64
c2_kernel_size = 3
c2_strides = 2
c2_padding = "SAME"

# pooling layer
p1_kernel_size = 2
p1_strides = 2
p1_padding = "VALID"
p1_n_fmaps = c2_n_fmaps

# fully connected
fc1_units = 64
outputs = 10

# Build graph
tf.reset_default_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32,shape=(None,n_inputs),name='X')
    X_reshaped = tf.reshape(X,shape=(-1,height,width,channels))
    y = tf.placeholder(tf.int32,shape=(None),name='y')
    
c1 = tf.layers.conv2d(X_reshaped,filters=c1_n_fmaps,kernel_size=c1_kernel_size,strides=c1_strides,padding=c1_padding,
                     activation=tf.nn.relu,name='C1')
c2 = tf.layers.conv2d(c1,filters=c2_n_fmaps,kernel_size=c2_kernel_size,strides=c2_strides,padding=c2_padding,
                     activation=tf.nn.relu,name='C2')
p1 = tf.nn.max_pool(c2,ksize=[1,p1_kernel_size,p1_kernel_size,1],strides=[1,p1_strides,p1_strides,1],padding=p1_padding,
                   name="p1")

# fully connected layer fc1
with tf.name_scope("fc1"):
    p1_flattened = tf.reshape(p1,shape=[-1,p1_n_fmaps*7*7])
    fc1 = tf.layers.dense(p1_flattened,fc1_units,activation=tf.nn.relu,name='fc1')

# fully connected output layer
with tf.name_scope('outputs'):
    logits = tf.layers.dense(fc1,outputs,name='logits')
    y_proba = tf.nn.softmax(logits,name='y_proba')
    
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=y)
    loss = tf.reduce_mean(xentropy,name='loss')
    
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits,y,1)
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32),name='accuracy')
    
with tf.name_scope('init_and_save'):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

#### Extract MNIST data

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


#### Train the convNet

In [4]:
n_epochs = 10
batch_size = 100

with tf.Session() as sess:
    init.run()
    n_batches = mnist.train.num_examples // batch_size
    for epoch in range(n_epochs):
        for iteration in range(n_batches):
            x_batch,y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op,feed_dict={X:x_batch,y:y_batch})
        accu_train = accuracy.eval(feed_dict={X:x_batch,y:y_batch})
        accu_test  = accuracy.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels})
        print(epoch,"Training accuracy:",accu_train,"Test accuracy:",accu_test)
        
        save_path = saver.save(sess, "./my_mnist_model")
        
    save_path = saver.save(sess, "./my_mnist_model_final")

0 Training accuracy: 0.98 Test accuracy: 0.9812
1 Training accuracy: 0.99 Test accuracy: 0.9865
2 Training accuracy: 1.0 Test accuracy: 0.9875
3 Training accuracy: 1.0 Test accuracy: 0.9877
4 Training accuracy: 1.0 Test accuracy: 0.9877
5 Training accuracy: 1.0 Test accuracy: 0.9885
6 Training accuracy: 1.0 Test accuracy: 0.9867
7 Training accuracy: 0.99 Test accuracy: 0.9882
8 Training accuracy: 1.0 Test accuracy: 0.9872
9 Training accuracy: 1.0 Test accuracy: 0.9898


#### Conclusion

We have trained a simple convolutional neural network to classify handwritten digits from MNIST dataset with 98.98% accuracy. Better performance can still be achieved with more complex CNNs such as AlexNet, GoogLeNet or ResNet